In [6]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import multiprocessing
#from multiprocessing import Process
#print("Number of cpu : ", multiprocessing.cpu_count())



Number of cpu :  16


In [2]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config.json')
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
for index,scan in scandf3.iterrows():
    scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
    if 'align-rccmdpath' in scan['processingstate'] and scan['rcboxpath'].is_file():
        if not 'commandlist_reconstructsettings' in scan['processingstate']:
            scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
        if not 'reconstruct-rccmdpath' in scan['processingstate'] :
            scan['commandlist_reconstruct'] = []
            scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + scan['rcboxpath'])  
            scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            scan['commandlist_reconstruct'].append('-save')
            scan['commandlist_reconstruct'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )

AttributeError: 'str' object has no attribute 'isin'

In [ ]:

def process_results(results):
  # This function will process the results of the GPU-intensive subprocess
  print(f'Processing results: {results}')

def cpu_intensive_function():
  # This function runs the CPU-intensive subprocess
  subprocess.run(['cmd.exe', '/C', 'findstr', 'test', 'large_file.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

# Create a client and connect to the cluster
client = ippclient.Client()
client.block = True

# Create a direct view to the engines
direct_view = client[:]

# Set up the parallel execution
with direct_view.sync_imports():
  import numpy as np

# Create a separate process for the CPU-intensive function
cpu_process = ippcontext.Client().load_balanced_view().apply(cpu_intensive_function)

# Run the GPU-intensive function in a loop
for i in range(10):
  # Run the GPU-intensive function on the engines
  result = direct_view.apply(gpu_intensive_function)

  # Process the results in the background
  process_results(result)

  # Sleep for a short period of time to simulate a long-running process
  time.sleep(1)

# Wait for the CPU-intensive process to finish
cpu_process.wait()

# Disconnect from the cluster
client.close()